In [186]:
import pandas as pd
import numpy as np
import scipy as sp

# Лабораторная работа №3
## Задание 3
## Кондрашин Тимофей 211-365 Вариант 11 (27.04.2023)

### Данные

Для заданного интервального выборочного ряда проверить гипотезу: закон распределения генеральной совокупности является показательным при уровне значимости $\alpha=0.05$

In [187]:
data = pd.DataFrame([38, 37, 40, 41, 43, 42, 31, 45, 34, 24])
n = len(data)
a = 0.05
min_x = 8.4
h = 2.2

data.T

,0,1,2,3,4,5,6,7,8,9
0,38,37,40,41,43,42,31,45,34,24


In [188]:
print(f'n = {n}, a = {a}, min_x = {min_x}, h = {h}')

n = 10, a = 0.05, min_x = 8.4, h = 2.2


## Шаг 1: Формулировка гипотезы

$H_0$: закон распределения генеральной совокупности является равномерным.
$H_1$: закон распределения генеральной совокупности не является равномерным.

## Шаг 2: Вычисление характеристик выборки.

### Вычисление среднего значения

$$\overline{x} = \frac{1}{n} \sum_{i=1}^{n} x_i$$

In [189]:
mean = data.mean()
mean

0    37.5
dtype: float64

### Вычисление дисперсии

$$s^2 = \frac{1}{n-1} \sum_{i=1}^{n} (x_i - \overline{x})^2$$

In [190]:
variance = data.var()
variance

0    40.277778
dtype: float64

### Вычисление стандартного отклонения

$$s = \sqrt{s^2}$$

In [191]:
std = data.std()
std

0    6.346478
dtype: float64

## Шаг 3: Проверка гипотезы о нормальности распределения

Для проверки гипотезы о нормальности распределения воспользуемся критерием согласия хи-квадрат. Сначала необходимо разбить интервальный ряд на интервалы.

Для этого найдём количество интервалов:

$$k = \lceil \log_2 n + 1 \rceil$$

In [192]:
k = int(np.ceil((data.max()[0] - data.min()[0]) / h))
k

10

Теперь можно разбить интервальный ряд на интервалы:

$$\Delta = \frac{max(x) - min(x)}{k}$$

In [193]:
delta = (data.max()[0] - data.min()[0]) / h
delta

9.545454545454545

Теперь можно построить интервальный ряд:

In [194]:
intervals = []
for i in range(k):
    intervals.append([min_x + i * delta, min_x + (i + 1) * delta])
intervals

[[8.4, 17.945454545454545],
 [17.945454545454545, 27.490909090909092],
 [27.490909090909092, 37.03636363636363],
 [37.03636363636363, 46.58181818181818],
 [46.58181818181818, 56.127272727272725],
 [56.127272727272725, 65.67272727272727],
 [65.67272727272727, 75.21818181818182],
 [75.21818181818182, 84.76363636363637],
 [84.76363636363637, 94.30909090909091],
 [94.30909090909091, 103.85454545454546]]

Далее, необходимо вычислить выборочные частоты попадания значений в каждый интервал:

$$p_i = \frac{n_i}{n}$$

In [195]:
p = []
for i in range(k):
    p.append(len(data[(data[0] >= min_x + i * delta) & (data[0] < min_x + (i + 1) * delta)]) / n)
p

[0.0, 0.1, 0.3, 0.6, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

Затем нужно вычислить теоретические (ожидаемые) частоты попадания значений в каждый интервал, если бы распределение было нормальным. Для этого воспользуемся формулой:
$$P_i = \Phi \left( \frac{x_{i+1} - \overline{x}}{s} \right) - \Phi \left( \frac{x_{i} - \overline{x}}{s} \right)$$

In [201]:
P = []
for i in range(k):
    P.append(sp.stats.norm.cdf((intervals[i][1] - mean) / std) - sp.stats.norm.cdf((intervals[i][0] - mean) / std))
P

[array([0.00102869]),
 array([0.05635419]),
 array([0.41349637]),
 array([0.45290357]),
 array([0.07454748]),
 array([0.00166292]),
 array([4.51513421e-06]),
 array([1.39778156e-09]),
 array([4.76285678e-14]),
 array([0.])]

In [202]:
intervals_t = pd.DataFrame(intervals, columns=['min', 'max'])
p_t = pd.DataFrame(p, columns=['p (выборочные частоты)'])
P_t = pd.DataFrame(P, columns=['P (ожидаемые частоты)'])
intervals_t.join(p_t).join(P_t).T

,0,1,2,3,4,5,6,7,8,9
min,8.400000,17.945455,27.490909,37.036364,46.581818,56.127273,65.672727,7.521818e+01,8.476364e+01,94.309091
max,17.945455,27.490909,37.036364,46.581818,56.127273,65.672727,75.218182,8.476364e+01,9.430909e+01,103.854545
p (выборочные частоты),0.000000,0.100000,0.300000,0.600000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000
P (ожидаемые частоты),0.001029,0.056354,0.413496,0.452904,0.074547,0.001663,0.000005,1.397782e-09,4.762857e-14,0.000000


## Шаг 4: Вычисление значения статистики критерия

Вычислим значение статистики критерия $\chi^2$:

$$\chi^2 = \sum_{i=1}^{k} \frac{(p_i - P_i)^2}{P_i}$$

In [203]:
chi2 = (((np.array(p) - np.array(P)) ** 2).sum()) / np.array(P).sum()
chi2

6.448397508760504

## Шаг 5: Определение критического значения и принятие решения

#### Уровень значимости:
a = 0.05

In [204]:
a

0.05

#### Степени свободы:
$$df = k - 1$$
где
- $k$ - количество интервалов,
- $p$ - количество параметров распределения, которые были оценены по выборке.

In [205]:
df = k - 1
df

9

Таблица распределения хи-квадрат ($\chi^2$) со степенями свободы 9 и уровнем значимости 0.05 даёт критическое значение $\chi^2 = 16.919$.

Так как вычисленное значение статистики критерия Пирсона ($\chi^2 = 6.448397508760504) не превышает критическое значение ($\chi^2 = 16.919$), то мы принимаем нулевую гипотезу $H_0$ и отвергаем альтернативную гипотезу $H_1$. Это означает, что закон распределения генеральной совокупности не является равномерным.


## Шаг 5: Вывод

Таким образом, на основе данных выборки мы можем с уверенностью утверждать, что закон распределения генеральной совокупности не является равномерным.